In [46]:
!pip install keras-tuner -q

In [47]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import Hyperband

In [48]:
# create model
def build_model(hp):
  model=keras.Sequential()
  # search num conv
  num_conv_layers = hp.Int('num_conv_layers', min_value=3, max_value=5, step=1)
  for i in range(num_conv_layers):
    # search num filtwe
    model.add(layers.Conv2D(
        filters=hp.Int(f'conv_{i}_filters', min_value=32, max_value=256, step=32),
        kernel_size=3,
        activation='relu'
    ))
    if i%2==0:
      model.add(layers.MaxPooling2D(pool_size=2))
    model.add(layers.Dropout(rate=0.2))


  model.add(layers.Flatten())
  # search num dense
  num_dense_layer=hp.Int('num_dense_layer',min_value=3, max_value=5, step=1)
  for i in range(num_dense_layer):
    # search num nerouns
    model.add(layers.Dense(
        units=hp.Int(f'dense_{i}_units', min_value=32, max_value=256, step=32),
        activation='relu'
    ))
  model.add(layers.Dense(10, activation='softmax'))

  # search learning rate
  optimizer = keras.optimizers.Adam(
      learning_rate=hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
  )
  model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model


In [49]:
# load dataset
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()

In [50]:
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)

In [51]:
import numpy as np
x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)

In [52]:
print(x_train.shape)

(60000, 28, 28, 1)


In [55]:
tuner = Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=5,
    executions_per_trial=2,
    overwrite=True,
    directory='tuner_dir',
    project_name='mnist_model'
)


In [56]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Trial 9 Complete [00h 02m 48s]
val_accuracy: 0.5418500006198883

Best val_accuracy So Far: 0.9918000102043152
Total elapsed time: 00h 11m 30s


In [64]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28,1))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 96)        27744     
                                                                 
 dropout_1 (Dropout)         (None, 11, 11, 96)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 96)          83040     
                                                        

In [63]:
tuner.results_summary()

Results summary
Results in tuner_dir/mnist_model
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0005 summary
Hyperparameters:
num_conv_layers: 3
conv_0_filters: 32
conv_1_filters: 96
conv_2_filters: 96
num_dense_layer: 3
dense_0_units: 32
dense_1_units: 128
dense_2_units: 64
learning_rate: 0.001
conv_3_filters: 128
conv_4_filters: 96
dense_3_units: 32
dense_4_units: 160
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0003
Score: 0.9918000102043152

Trial 0003 summary
Hyperparameters:
num_conv_layers: 3
conv_0_filters: 32
conv_1_filters: 96
conv_2_filters: 96
num_dense_layer: 3
dense_0_units: 32
dense_1_units: 128
dense_2_units: 64
learning_rate: 0.001
conv_3_filters: 128
conv_4_filters: 96
dense_3_units: 32
dense_4_units: 160
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9878000020980835

Trial 0002 summary
Hyperparameters:
num_conv_layers: 4
conv_0_filters: 64
conv_1_filters: 32
con